# 1) Cargar la data

In [1]:
import os, pickle
import pandas as pd
from tabulate import tabulate
from rich import print

In [2]:
def leer_nombre_ficheros(elementos_SQ, temporalidades):
    """
    Encuentra los elementos anteriores al primer match entre la lista de elementos_SQ y temporalidades.

    Args:
        elementos_SQ (list): Lista de elementos a verificar.
        temporalidades (list): Lista de temporalidades a buscar.

    Returns:
        list: Lista de listas que contienen activos encontrados dentro del elemento en elementos_SQ.
        list: Lista de listas que contienen las temporalidades encontradas en cada elemento de elementos_SQ.
    """
    id_busqueda = []
    activos_encontrados = []
    temporalidades_encontradas = []
    
    for elemento in elementos_SQ:
        elementos_previos = []
        primer_elemento = elemento.split('_')[0]
        
        for temporalidad in temporalidades:
            index = elemento.find(temporalidad)
            if index != -1:
                elementos_previos.extend(elemento[:index].split('_')[1:-1])
                temporalidades_encontradas.append(elemento[index:].split('_')[:-1])
                break
        
        id_busqueda.append(primer_elemento)
        activos_encontrados.append(elementos_previos)
    
    return id_busqueda, activos_encontrados, temporalidades_encontradas

# DECORADOR
def styler(func):
    def wrapper(*args, **kwargs):
        df = func(*args, **kwargs)
        # formato para la impresion con estilo
        keys= list(df.columns)
        values = ['{:.2f}' for i in range(len(keys))]
        format_dict = dict(zip(keys, values))
        return df.style.format(format_dict).background_gradient(cmap='PRGn')
    return wrapper

# SE UTILIZA PARA VISUALIZAR LOS DATOS DEL OBJETO correlaciones

@styler
def get_correlation(data: dict, activo: str):
    """
    Devuelve la matriz de correlaciones para un activo determinado.

    Args:
        data (dict): Diccionario con los datos de correlaciones de todos los activos.
        activo (str): Activo a buscar en el diccionario.

    Returns:
        DataFrame: Matriz de correlaciones para el activo.

    ACLARACION: El DataFrame es tipo Styler. Por lo que algunas funcionalidades propias de los DataFrame se pierden.
    """
    # Filter the data dictionary based on the activo
    filtered_data = {
        key: values[activo.upper()]
        for key, values in data.items()
    }
    # Create the DataFrame
    df = pd.DataFrame(filtered_data)
    df.columns.name = 'correlacion {}'.format(activo)
    return df

def crear_menu(id_search, activos, temporalidades):
    menu = ""

    for i in range(len(id_search)):
        assets = ' - '.join(activos[i])
        temps = ' '.join(temporalidades[i])
        menu_item = f'{id_search[i]}) {assets} : {temps}\n'
        menu += menu_item

    return menu


In [3]:
# PROGRAMA PRINCIPAL
# Load the data
date_path = os.path.join(os.getcwd(), 'data')

# Get the date directory
directorios = os.listdir(date_path)
print(f' Directorios guardados: {directorios}')

directorios_elegido = directorios[-1]#TODO: acá va el input para cargar los datos correspondientes
file_dir = os.path.join(date_path, directorios_elegido)#TODO: estamos eligiendo el ultimo directorio, HARDCODED es la carpeta del dia de hoy
print(f' Directorio elegido: {directorios_elegido}')
#TODO: APLICAR crear_menu para elegir el directorio a cargar
print(f' Cantidad de solicitudes guardadas: {len(os.listdir(file_dir))//4}')#WARNING: Cuando hubo errores en la descarga //4 puede arrojar errores más adelante

# obtener los nombres de los activos descargados
environment_temporalities = ['1M','1w','3d','1d','12h', '6h','4h','2h','1h','30m','15m','5m','3m', '1m']
elementos_SQ = [file for file in os.listdir(file_dir) if file.endswith('SQ.pickle')]

id_search, activos_encontrados, temporalidades = leer_nombre_ficheros(elementos_SQ, environment_temporalities)


if __name__ == '__main__':
    #TODO: crear una funcion main() que tenga argumentos de entrada, la fecha y luego los activos disponibles
    print(crear_menu(id_search, activos_encontrados, temporalidades))

    id_value = 0
    id_elegido = id_search[id_value]#TODO: acá va el input para cargar los datos correspondientes
    print(f' ID elegido: {id_elegido}')
    activos_elegidos = activos_encontrados[id_value]
    temporalidades_correspondientes = temporalidades[id_value]

    print(f' Activos elegidos: {activos_elegidos}')
    print(f' Temporalidades correspondientes: {temporalidades_correspondientes}')

    archivos_necesarios = ['.pickle','_SQ.pickle', '_CORR.pickle', '_TW.pickle']
    file_name_without_extension = f'{id_elegido}_' + '_'.join(activos_elegidos)+'_'+'_'.join(temporalidades_correspondientes)
    loaded_files = []
    for extension in archivos_necesarios:
        file_name = file_name_without_extension + extension
        file_path = os.path.join(file_dir, file_name)
        with open(file_path, 'rb') as file:
            loaded_files.append(pickle.load(file))
    data = {
        'data_raw': loaded_files[0],
        'data_SQ': loaded_files[1],
        'SQ_CORR': loaded_files[2],
        'time_window': loaded_files[3]
    }

Directorios guardados: ['04_junio_2023', '05_junio_2023', '06_junio_2023', '07_junio_2023', '08_junio_2023', 
'09_junio_2023', '10_junio_2023', '11_junio_2023', '16_junio_2023']

Directorio elegido: 16_junio_2023

Cantidad de solicitudes guardadas: 1

1) BTC - XRP - ETH : 4h 2h 1h 30m 15m 5m 3m

ID elegido: 1

Activos elegidos: ['BTC', 'XRP', 'ETH']

Temporalidades correspondientes: ['4h', '2h', '1h', '30m', '15m', '5m', '3m']

# 2) Mostrar las correlaciones

Podría utilizarse la libreria Rich para armar tablas y exportarlas en formato txt


In [4]:
if __name__ == '__main__':
    asset = 'BTC'

    df = get_correlation(data['SQ_CORR'], asset)
    display(df)

correlacion BTC,9,12,14,18,20,22,30,40,50,60,70,80,90
4h,0.50,0.64,0.48,0.55,0.55,0.54,0.50,0.49,0.53,0.28,0.29,0.46,0.57
2h,0.54,0.57,0.44,0.48,0.50,0.51,0.56,0.53,0.50,0.18,0.21,0.35,0.41
1h,0.61,0.37,0.48,0.54,0.55,0.56,0.59,0.43,0.56,0.52,0.52,0.45,0.30
30m,0.99,0.99,0.72,0.60,0.60,0.61,0.68,0.54,0.56,0.50,0.53,0.50,0.48
15m,0.85,0.95,0.98,0.79,0.77,0.74,0.61,0.57,0.61,0.48,0.51,0.53,0.55
5m,-0.05,-0.05,0.33,0.61,0.68,0.74,0.89,0.81,0.79,0.65,0.62,0.58,0.54
3m,-0.16,-0.08,-0.01,0.24,0.27,0.31,0.49,0.80,0.90,0.68,0.75,0.79,0.84


# 4) Visualización de la data

COMENTARIOS RESPECTO A LOS VALORES DE LAS MEDIAS DEL PRECIO

- SMA: MEDIA MOVIL QUE PONDERA TODOS LOS PRECIOS DE CIERRE POR IGUAL
- TRIMA: MEDIA MOVIL CONSTRUIDA A PARTIR DE LA SMA Y LA SMA DE LA SMA
- EMA: MEDIA MOVIL QUE PONDERA DE FORMA EXPONENCIALOS LOS PRECIO DE CIERRE
- DEMA: MEDIA MOVIL CONSTRUIDA A PARTIR DE LA EMA Y LA EMA DE LA EMA
- TEMA: MEDIA MOVIL CONTRUIDA A PARTIR DE LA EMA Y LA EMA DE LA EMA Y LA EMA DE LA EMA DE LA EMA
- WMA: MEDIA MOVIL PONEDARADA DE FORMA LINEAL RESPECTO AL PRECIO DE CIERRE


ChatGPT:

Ambas medias móviles, DEMA y TEMA, son consideradas indicadores más sensibles a los cambios de tendencia en comparación con las medias móviles más comunes, como la SMA y la EMA. Sin embargo, es importante recordar que ninguna media móvil es infalible y todas tienen sus ventajas y desventajas. Es recomendable probar diferentes tipos de medias móviles y combinarlas con otros indicadores y análisis para obtener una visión más completa de la situación del mercado.

In [5]:
# Una forma auxiliar de imprimir dataframes
# La previsualización de los datos asociados a los axis tienen que ser visualizados de esta manera

import plotly.graph_objects as go
import plotly.subplots as ms
from rich import print
from datetime import datetime
from screener.medias_moviles import get_moving_averages
#print(f'Correlación {asset} con Squeeze Momentum Indicator')
#print(df.loc[:,70:])
#print(df.loc[:,30:60])
#print(df.loc[:,9:22])


In [6]:
def create_multiple_cs_plot(data, asset, temp, periods, add_mvg_avg=False, mvg_avg_period=200, remove_candlestick=False, add_volume=False):
    """Función que se utiliza para crear multiples graficos en función de una temporalidad y varios periodos tomados de momentum (len)
    
    [description]

    [args]
    data (dict): diccionario con los datos de los activos
    asset (str): nombre del activo
    temp (str): temporalidad
    periods (list): lista de periodos a graficar

    [return]
    None
    """
    for period in periods:
        df = data['data_SQ'][period][asset][temp]
        corr = data['SQ_CORR'][period][asset][temp]
        # FILTRO DE LONGITUD DE DATOS
        try:
            period_window = data['time_window'][temp][period]
        except:
            period_window = df.index[0]
        candlestick_squeeze_plot(df, asset, temp, period, corr, period_window, add_mvg_avg=add_mvg_avg, mvg_avg_period=mvg_avg_period, remove_candlestick=remove_candlestick, add_volume=add_volume)

def candlestick_squeeze_plot(data, asset, temp, period, corr, period_window, add_mvg_avg=False, mvg_avg_period=200, remove_candlestick=False, add_volume=False):
    """
    Funcion que crea un grafico de velas con un squeeze plot en graficos separados

    """


    features = ['open', 'close', 'high', 'low','volume', 'value']
    df = data.loc[period_window:,features]
    if add_mvg_avg:
        df_mvg_avg = get_moving_averages(data, 'close', mvg_avg_period)
        # display(df_mvg_avg)
        df_mvg_avg = df_mvg_avg.loc[:,[f'EMA_{mvg_avg_period}', f'TRIMA_L_{mvg_avg_period}']]
        # display(df_mvg_avg)
        df_mvg_avg_10 = get_moving_averages(data, 'close', 10)
        # display(df_mvg_avg_10)
        df_mvg_avg_10 = df_mvg_avg_10.loc[:, ['EMA_10', 'WMA_10', 'DEMA_10', 'TEMA_10']]
        # display(df_mvg_avg_10)
        df_mvg_avg = df_mvg_avg.loc[period_window:, :]
        # display(df_mvg_avg)
        df_mvg_avg_10 = df_mvg_avg_10.loc[period_window:, :]
        # display(df_mvg_avg_10)
        scatters_mvg_avg = []
        for col in df_mvg_avg.columns:
            scatters_mvg_avg.append(go.Scatter(x=df_mvg_avg.index, y=df_mvg_avg[col], name=col))
        scatters_mvg_avg_10 = []
        for col in df_mvg_avg_10.columns:
            scatters_mvg_avg_10.append(go.Scatter(x=df_mvg_avg_10.index, y=df_mvg_avg_10[col], name=col))

    dist_tick = len(df)//7
    candlestick = go.Candlestick(x=df.index,
                    open=df['open'],
                    high=df['high'],
                    low=df['low'],
                    close=df['close'])

    bar_squeeze = go.Bar(x=df.index, y=df['value'], name='squeeze')
    

    if add_volume:
        bar_volume = go.Bar(x=df.index, y=df['volume'], name='volume', marker_color='blue')
        fig = ms.make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02, row_heights=[0.8, 0.2])
        if add_mvg_avg and not remove_candlestick:
            fig.add_trace(candlestick, row=1, col=1)
            for scatter in scatters_mvg_avg:
                fig.add_trace(scatter, row=1, col=1)
            for scatter in scatters_mvg_avg_10:
                fig.add_trace(scatter, row=1, col=1)
        elif add_mvg_avg and remove_candlestick:
            for scatter in scatters_mvg_avg:
                fig.add_trace(scatter, row=1, col=1)
            for scatter in scatters_mvg_avg_10:
                fig.add_trace(scatter, row=1, col=1)
        else:
            fig.add_trace(candlestick, row=1, col=1)

        fig.add_trace(bar_volume, row=2, col=1)
        
        

    # CREACION DE LAS FIGURES
    else:
        if add_mvg_avg and not remove_candlestick:
            fig = go.Figure(data=[candlestick, *scatters_mvg_avg, *scatters_mvg_avg_10])
        elif add_mvg_avg and remove_candlestick:
            fig = go.Figure(data=[*scatters_mvg_avg, *scatters_mvg_avg_10])
        else:
            fig = go.Figure(data=[candlestick])
    
    
    
    fig2 = go.Figure(data=[bar_squeeze])


    fig.layout.xaxis.type = 'category'
    fig.layout.xaxis.rangeslider.visible = False
    fig.update_layout(
        xaxis = dict(
            tickmode = 'array',
            tickvals = df.index[dist_tick//2::dist_tick],#TODO:considerar este slide para balancear los ticks del eje x
            ticktext = df.index[:-dist_tick//2:dist_tick],#TODO:considerar este slide para balancear los ticks del eje x
        ),
        autosize=False,
        width=1200,
        height=600,
        margin=dict(
            
            l=50,
            r=50,
            b=50,
            t=50,
            pad=4
        ),
        title=f'{asset} {temp} | len squeeze: {period} | corr: {corr}',
        showlegend=False
    )
    if asset == 'BTC':
        fig.update_layout(
        paper_bgcolor="#B0BEC5",
        plot_bgcolor="#BDBDBD",
        )

    else:
        fig.update_layout(
        paper_bgcolor="LightSteelBlue",
        )

    fig2.update_layout(
        autosize=False,
        width=1200,
        height=300,
        margin=dict(
            
            l=50,
            r=50,
            b=1,
            t=10,
            pad=4
        )
    )
    if asset == 'BTC':
        fig2.update_layout(
        paper_bgcolor="#B0BEC5",
        plot_bgcolor="#BDBDBD",
        )

    else:
        fig2.update_layout(
        paper_bgcolor="LightSteelBlue")

    fig.show()
    fig2.show() 


In [7]:
if __name__ == '__main__':
    asset = 'BTC'

    df = get_correlation(data['SQ_CORR'], asset)
    display(df)

correlacion BTC,9,12,14,18,20,22,30,40,50,60,70,80,90
4h,0.50,0.64,0.48,0.55,0.55,0.54,0.50,0.49,0.53,0.28,0.29,0.46,0.57
2h,0.54,0.57,0.44,0.48,0.50,0.51,0.56,0.53,0.50,0.18,0.21,0.35,0.41
1h,0.61,0.37,0.48,0.54,0.55,0.56,0.59,0.43,0.56,0.52,0.52,0.45,0.30
30m,0.99,0.99,0.72,0.60,0.60,0.61,0.68,0.54,0.56,0.50,0.53,0.50,0.48
15m,0.85,0.95,0.98,0.79,0.77,0.74,0.61,0.57,0.61,0.48,0.51,0.53,0.55
5m,-0.05,-0.05,0.33,0.61,0.68,0.74,0.89,0.81,0.79,0.65,0.62,0.58,0.54
3m,-0.16,-0.08,-0.01,0.24,0.27,0.31,0.49,0.80,0.90,0.68,0.75,0.79,0.84


In [10]:
#4h len 12, 22

asset = 'BTC'
temp = '4h'
periods = [60, 50, 40, 30, 14, 12]
create_multiple_cs_plot(data, asset, temp, periods, add_mvg_avg=True, mvg_avg_period=55, remove_candlestick=False, add_volume=True)


In [11]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Crear las figuras y subplots
fig = make_subplots(rows=2, cols=2, shared_xaxes=True, shared_yaxes=True)

# Agregar trazados a los subplots
fig.add_trace(go.Scatter(x=[1, 2, 3], y=[1, 3, 2]),
              row=1, col=1)

fig.add_trace(go.Scatter(x=[1, 2, 3], y=[2, 1, 3]),
              row=1, col=2)

fig.add_trace(go.Scatter(x=[1, 2, 3], y=[3, 2, 1]),
              row=2, col=1)

fig.add_trace(go.Scatter(x=[1, 2, 3], y=[1, 2, 3]),
              row=2, col=2)

# Actualizar las opciones de layout para mostrar la herramienta box
fig.update_layout(
    updatemenus=[
        dict(
            buttons=[
                dict(
                    args=[{"box": {"visible": True}}],
                    label="Box",
                    method="update"
                ),
                dict(
                    args=[{"box": {"visible": False}}],
                    label="Reset",
                    method="update"
                )
            ],
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.1,
            yanchor="top"
        ),
    ]
)

# Mostrar el gráfico
fig.show()


In [67]:
asset = 'BTC'
temp = '1h'
periods = [50, 18, 14, 12, 9]

create_multiple_cs_plot(data, asset, temp, periods,add_mvg_avg=True, mvg_avg_period=55, remove_candlestick=False, add_volume=True)

In [41]:
asset = 'XRP'
temp = '30m'
periods = [70, 40, 22]

create_multiple_cs_plot(data, asset, temp, periods, add_mvg_avg=True, mvg_avg_period=200, remove_candlestick=False, add_volume=True)

In [43]:
asset = 'ETH'
temp = '5m'
periods = [9,12,14,18, 20, 22, 30, 50]

create_multiple_cs_plot(data, asset, temp, periods, add_mvg_avg=True, mvg_avg_period=55, remove_candlestick=False, add_volume=True)

# 5) Exportar tablas, registros y visualizaciones